In [27]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from pathlib import Path
# Step 1: Create a session and load the page
url1 = 'https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=0-2+Feet+-+Very+Tight&PerMode=Totals'
url2 = 'https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=2-4+Feet+-+Tight&PerMode=Totals'
url3 = 'https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=4-6+Feet+-+Open&PerMode=Totals'
url4 = 'https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=6%2B+Feet+-+Wide+Open&PerMode=Totals'
url_list = [url1,url2,url3,url4]
#url_list =[url +'&SeasonType=Playoffs' for url in url_list]
url_list =[url +'&SeasonType=Regular+Season'for url in url_list]

def get_tables(url_list):
    data = []
    xpath = '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[3]/table'
    options = webdriver.FirefoxOptions()
    driver = webdriver.Firefox(options=options)
    for url in url_list:
        
        driver.get(url)
        print(url)
        # Wait for the page to fully load
        driver.implicitly_wait(20)
        element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, xpath)))

        # Step 2: Parse HTML code and grab tables with Beautiful Soup
        soup = BeautifulSoup(driver.page_source, 'lxml')

        tables = soup.find_all('table')

        # Step 3: Read tables with Pandas read_html()
        dfs = pd.read_html(str(tables))

        #print(f'Total tables: {len(dfs)}')
        #print(dfs[2].head())

        
        df= dfs[-1]
        #print(df)
        drop = ['Unnamed: 16_level_1', 'Unnamed: 17_level_1', 'Unnamed: 18_level_1']
        df.columns = df.columns.droplevel()
        df = df.drop(columns = drop)
        data.append(df)
    driver.close()
    return data
#url_list = [url1]#
def multiyear_shooting(url_list,team_round=0,playoffs = True):
    df_list = []

    for i in range(2023,2024):
        year = i+1
        season = '&Season='+str(i)+'-'+str(i+1 - 2000)
        year_url = [url+season for url in url_list]
        if team_round!=0:
            year_url = [url + '&PORound='+str(team_round) for url in year_url]
        frames = get_tables(year_url)

 
        path = str(i+1)+'/playoff_shooting/round'+str(team_round)+'/'
        if playoffs == False:
            path = str(i+1)+'/team_shooting/'
        output_dir = Path(path)
        output_dir.mkdir(parents=True, exist_ok=True)
        #terms = ['data/teampullup.csv','data/teamcatchshoot.csv','data/teamundersix.csv','data/teamiso.csv','data/teamtransition.csv']
        terms = ['very_tight','tight','open','wide_open']
        
        
        for k in range(len(terms)):
            
            frames[k]['shot_coverage'] = terms[k]
            if playoffs == True:
                frames[k]['round'] = team_round
            frames[k]['year'] = year
        df = pd.concat(frames)
        df.to_csv(path+'team_shooting.csv',index = False)
        df_list.append(df)
    final_df = pd.concat(df_list)
    return final_df

df = multiyear_shooting(url_list,playoffs=False)

https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=0-2+Feet+-+Very+Tight&PerMode=Totals&SeasonType=Regular+Season&Season=2013-14
https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=2-4+Feet+-+Tight&PerMode=Totals&SeasonType=Regular+Season&Season=2013-14
https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=4-6+Feet+-+Open&PerMode=Totals&SeasonType=Regular+Season&Season=2013-14
https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=6%2B+Feet+-+Wide+Open&PerMode=Totals&SeasonType=Regular+Season&Season=2013-14
https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=0-2+Feet+-+Very+Tight&PerMode=Totals&SeasonType=Regular+Season&Season=2014-15
https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=2-4+Feet+-+Tight&PerMode=Totals&SeasonType=Regular+Season&Season=2014-15
https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=4-6+Feet+-+Open&PerMode=Totals&Season

,TEAM,GP,G,Freq%,FGM,FGA,FG%,eFG%,2FG Freq%,2FGM,2FGA,2FG%,3FG Freq%,3PM,3PA,3P%,shot_coverage,year
0,Detroit Pistons,82,82,23.1,843,1554,54.2,54.4,22.7,839,1529,54.9,0.4,4,25,16.0,very_tight,2014
1,Philadelphia 76ers,82,82,22.5,762,1479,51.5,51.7,22.2,756,1461,51.7,0.3,6,18,33.3,very_tight,2014
2,Minnesota Timberwolves,82,82,20.0,707,1342,52.7,52.8,19.8,705,1328,53.1,0.2,2,14,14.3,very_tight,2014
3,Indiana Pacers,82,82,21.0,673,1302,51.7,51.8,20.7,670,1283,52.2,0.3,3,19,15.8,very_tight,2014
4,Memphis Grizzlies,82,82,20.2,689,1276,54.0,54.0,20.0,688,1263,54.5,0.2,1,13,7.7,very_tight,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,Washington Wizards,82,82,19.9,594,1415,42.0,58.0,3.3,142,235,60.4,16.6,452,1180,38.3,wide_open,2023
26,Denver Nuggets,82,82,19.8,628,1406,44.7,61.2,3.7,164,264,62.1,16.1,464,1142,40.6,wide_open,2023
27,LA Clippers,82,82,19.7,615,1392,44.2,62.3,2.6,111,181,61.3,17.2,504,1211,41.6,wide_open,2023
28,Charlotte Hornets,82,82,18.8,567,1390,40.8,55.5,3.4,159,249,63.9,15.5,408,1141,35.8,wide_open,2023


In [28]:
df.to_csv('team_shooting.csv',index = False)